In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import soundfile as sf
from pathlib import Path
import re
import json
import os
import sys
# sys.path.append('..\soundbay')
# os.chdir('../../soundbay')
print(os.getcwd())
from utils.metadata_processing import (reorder_columns_to_default_view, 
                                       correct_call_times_with_duration,
                                       non_overlap_df, bg_from_non_overlap_calls)

C:\Users\amitg\Documents\GitHub\soundbay\soundbay


In [4]:
sys.path.append('C:/Users')
# selection_path = Path('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-08-12-67407878-RestingHole1A')
#recording_path = Path('/mnt/c/users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/170708155527')
selection_path = Path('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2')
# selection_path2 = Path('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A')
if not selection_path.exists():
    print("Path does not exist")
# all_files = list(selection_path.iterdir())
# print("All files in directory:", all_files)

filelist = list(selection_path.glob('*.txt'))
print("Filtered .txt files:", filelist)
# filelist2 = list(selection_path2.glob('*.txt'))

filelist_short = []
for file in filelist:
    name_ext = os.path.basename(file)
    name, txt = os.path.splitext(name_ext)
    # name = re.search(r'67\d{6,6}.\d{12,12}', file.as_posix()).group()
    # print(name)
    name = re.search(r'adjusted_\d+\.\d+', file.as_posix()).group()
    print(name)
    filelist_short.append(name)

Filtered .txt files: [WindowsPath('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2/adjusted_67670025.201212101523.Table.1.selections_bb.txt.txt'), WindowsPath('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2/adjusted_67670025.201212131511.Table.1.selections_bb.txt.txt'), WindowsPath('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2/adjusted_67670025.201212161507.Table.1.selections_bb.txt.txt'), WindowsPath('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2/adjusted_67670025.201212191505.Table.1.selections_bb.txt.txt'), WindowsPath('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/augmented_annotations2/adjusted_67670025.201212221503.Table.1.selections_bb.txt.txt')]
adjusted_67670025.201212101523.Table.1.selections_bb.txt
adjusted_67670025.201212101523
adjusted_67670025.201212131511.Table.1.select

In [5]:
filelist_short

['adjusted_67670025.201212101523',
 'adjusted_67670025.201212131511',
 'adjusted_67670025.201212161507',
 'adjusted_67670025.201212191505',
 'adjusted_67670025.201212221503']

In [17]:
textfile = open("ManateesList2.txt","w")
for f in filelist_short:
    textfile.write(f+"\n")
textfile.close()

In [8]:
metadata=[]
metadata2 = []
for file in filelist:
    dfTemp = pd.read_csv(file, sep="\t")
    # dfTemp['filename'] = re.search(r'17\d{10,10}', file.as_posix()).group() 
    dfTemp['filename'] = re.search(r'adjusted_\d+\.\d+', file.as_posix()).group().split('adjusted_')[1]
    dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
    dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    dfTemp['filepath'] = os.path.basename(selection_path)#os.path.join(,file,'.txt')
    metadata.append(dfTemp)
# for file in filelist2:
    # dfTemp = pd.read_csv(file, sep="\t")
    # dfTemp['filename'] = re.search(r'17\d{10,10}', file.as_posix()).group() 
    # dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
    # dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    # dfTemp['filepath'] = os.path.basename(selection_path2)#os.path.join(,file,'.txt')
    # metadata2.append(dfTemp)

metadata = pd.concat(metadata)
metadata.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)
# metadata2 = pd.concat(metadata2)
# metadata2.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)
# frames = [metadata,metadata2]
# PositiveCalls = pd.concat(frames, axis=0)
PositiveCalls = metadata
# PositiveCalls = PositiveCalls.drop(['Type of NLP','UpdatedNLP','SNR code','manatee','Manatee', 'MANATEE VOC',
                                    # 'mANATEE','2ndMeasure','AB Notes','AB notes'], axis=1)
PositiveCalls['label'] = np.ones((len(PositiveCalls),), dtype=int) #assuming everything is calls. TODO: filter out by notes


In [9]:
PositiveCalls

,Selection,View,Channel,begin_time,end_time,Low Freq (Hz),Delta Time (s),Delta Freq (Hz),High Freq (Hz),Inband Power (dB FS),Begin Date Time,SNR,NUM HARM,recording_filename,filename,StartMicInWater,EndMicInWater,filepath,label
0,1,Spectrogram 1,1,412.928222,413.186087,11902.439,0.2579,12682.927,24585.366,-93.32,2012/12/10 15:29:52.9282,3.3,2,6.767003e+07,67670025.201212101523,412.928222,10102.456150,augmented_annotations2,1
1,2,Spectrogram 1,1,415.595797,415.688457,3719.600,0.0927,10914.546,14634.146,-93.64,2012/12/10 15:29:55.5958,0.3,2,6.767003e+07,67670025.201212101523,412.928222,10102.456150,augmented_annotations2,1
2,3,Spectrogram 1,1,418.951454,419.321578,9560.976,0.3701,14243.902,23804.878,-92.90,2012/12/10 15:29:58.9515,4.1,3,6.767003e+07,67670025.201212101523,412.928222,10102.456150,augmented_annotations2,1
3,4,Spectrogram 1,1,463.320853,463.487096,1170.732,0.1662,6829.268,8000.000,-94.22,2012/12/10 15:30:43.3209,0.8,2,6.767003e+07,67670025.201212101523,412.928222,10102.456150,augmented_annotations2,1
4,5,Spectrogram 1,1,496.389546,496.454953,7414.634,0.0654,15798.481,23213.115,-94.09,2012/12/10 15:31:16.3895,0.0,1,6.767003e+07,67670025.201212101523,412.928222,10102.456150,augmented_annotations2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,706,Spectrogram 1,1,29058.097930,29058.506469,7400.881,0.4085,24951.542,32352.423,NaN,2012/12/11 11:26:59.1224,NaN,5,6.767003e+07,67670025.201212221503,12.793650,30584.991493,augmented_annotations2,1
205,707,Spectrogram 1,1,30020.115927,30020.347176,3171.806,0.2312,18185.022,21356.828,NaN,2012/12/11 11:43:01.1404,NaN,3,6.767003e+07,67670025.201212221503,12.793650,30584.991493,augmented_annotations2,1
206,708,Spectrogram 1,1,30339.506471,30339.791677,7189.427,0.2852,15859.031,23048.458,NaN,2012/12/11 11:48:20.5310,NaN,4,6.767003e+07,67670025.201212221503,12.793650,30584.991493,augmented_annotations2,1
207,709,Spectrogram 1,1,30413.926736,30414.204234,8246.696,0.2775,21145.374,29392.070,NaN,2012/12/11 11:49:34.9512,NaN,2,6.767003e+07,67670025.201212221503,12.793650,30584.991493,augmented_annotations2,1


In [10]:
PositiveCalls = non_overlap_df(PositiveCalls)
annotation_file = bg_from_non_overlap_calls(PositiveCalls)
annotation_file.to_csv('Manatee_annotations_placencia_august24_bg_from_calls.csv', index=False)